In [192]:
import numpy as np
import pandas as pd
import re
from datetime import datetime
import time

In [ ]:
%store -r boweryWorking
%store -r seatgeek

In [10]:
#pop to float
def pop_to_float(x):
    pop_to_int = []
    for i in x:
        try:
            i = float(i)
            pop_to_int.append(i)

        except:
            pop_to_int.append(i)
    return pop_to_int

In [49]:
# missing pop to 0
boweryWorking.artist2_popularity = np.where(boweryWorking['artist2_popularity'].notnull(), boweryWorking['artist2_popularity'], 0)
boweryWorking.artist_popularityO1 = np.where(boweryWorking['artist_popularityO1'].notnull(), boweryWorking['artist_popularityO1'], 0)
boweryWorking.artist_popularityO2 = np.where(boweryWorking['artist_popularityO2'].notnull(), boweryWorking['artist_popularityO2'], 0)

boweryWorking.artist2_followers = np.where(boweryWorking['artist2_followers'] == np.nan, boweryWorking['artist2_followers'], 0)
boweryWorking.artist_followersO1 = np.where(boweryWorking['artist_followersO1'] == np.nan, boweryWorking['artist_followersO1'], 0)
boweryWorking.artist_followersO2 = np.where(boweryWorking['artist_followersO2'] == np.nan, boweryWorking['artist_followersO2'], 0)





In [50]:
boweryWorking['total_pop'] = boweryWorking['artist_popularity'] + boweryWorking['artist2_popularity'] + boweryWorking['artist_popularityO1'] + boweryWorking['artist_popularityO2']
boweryWorking['total_fol'] = boweryWorking['artist_followers'] + boweryWorking['artist2_followers'] + boweryWorking['artist_followersO1'] + boweryWorking['artist_followersO2']




In [69]:
# day scrape to show as float
def scrapeNum(x):
    num1 = []
    for i in x:
        m = re.search("^\d+", str(i))
        num1.append(m.group(0))
    return num1

In [71]:
boweryWorking.days_scrape_to_show = scrapeNum(boweryWorking.days_scrape_to_show)

In [75]:
# make primary price column which is adv price high, else door price, else adv price low
boweryWorking['primary_price'] = np.where(boweryWorking['adv_price_high'].notnull(), 
                                          boweryWorking['adv_price_high'], 
                                          np.where(boweryWorking['door_price'].notnull(), 
                                                            boweryWorking['door_price'], 
                                                            boweryWorking['adv_price_low']))

In [80]:
boweryData = boweryWorking.drop(['door_price', 'adv_price_low', 'adv_price_high', 
                          'artist2_followers',  'artist2_popularity',
                          'artist_followersO1', 'artist_popularityO1',
                          'artist_followersO2',  'artist_popularityO2'
                         ], axis=1, inplace=False)

In [176]:
# clean concert venues for merge
# drop venues with less than 4 shows
# fete music hall combine ballroom and lounge
# royale to royale boston...etc

In [98]:
venueGroups = boweryWorking.groupby('venue')['venue']
venueGroups  = venueGroups.size()
dropVenues = pd.DataFrame(venueGroups[venueGroups<4])

In [149]:
venues = list(dropVenues.index)

In [144]:
boweryData = boweryData[~boweryData['venue'].isin(venues)]

In [153]:
vg = boweryData.groupby('venue')
vg.size()

venue
Aisle 5                        5
Brooklyn Steel                43
Fête Music Hall - Ballroom     5
Fête Music Hall - Lounge       4
Gasa Gasa                     17
Great Scott                   50
Music Hall of Williamsburg    38
Rams Head Live!               22
Rough Trade NYC               44
Royale                        19
Saturn                        27
Starland Ballroom              6
Terminal 5                     7
Terminal West                  9
The EARL                       6
The National                  31
The NorVa                     28
The Sinclair                  55
Underground Arts              14
Variety Playhouse              7
dtype: int64

In [172]:
# make venue name the same for merge
mergeVenues = np.where(boweryData['venue'] == "Fête Music Hall - Ballroom", "Fete Music Hall", 
         np.where(boweryData['venue'] == "Fête Music Hall - Lounge", "Fete Music Hall", 
                 np.where(boweryData['venue'] == "Rams Head Live!", "Rams Head Live",
                         np.where(boweryData['venue'] == "Royale", "Royale Boston",
                                  np.where(boweryData['venue'] == "The EARL", "The Earl",
                                           np.where(boweryData['venue'] == "The norva", "The Norva", boweryData['venue']
                                 ))))))

In [173]:
boweryData.venue = mergeVenues

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [175]:
vg = boweryData.groupby('venue')
vg.size()

venue
Aisle 5                        5
Brooklyn Steel                43
Fete Music Hall                9
Gasa Gasa                     17
Great Scott                   50
Music Hall of Williamsburg    38
Rams Head Live                22
Rough Trade NYC               44
Royale Boston                 19
Saturn                        27
Starland Ballroom              6
Terminal 5                     7
Terminal West                  9
The Earl                       6
The National                  31
The Norva                     28
The Sinclair                  55
Underground Arts              14
Variety Playhouse              7
dtype: int64

In [154]:
SGvenues = seatgeek.groupby('venue')
SGvenues.size()

venue
Aisle 5                       43
Brooklyn Steel                61
Fete Music Hall               12
Gasa Gasa                     23
Great Scott                   30
Music Hall of Williamsburg    37
Rams Head Live                30
Rough Trade NYC               45
Royale Boston                 36
Saturn                        39
Starland Ballroom             15
Terminal 5                    15
Terminal West                 40
The Earl                       8
The National                  44
The Norva                     33
The Sinclair                  65
Underground Arts              26
Variety Playhouse             33
dtype: int64

In [194]:
from dateutil.parser import parse
def parse_time(x):
    empty_list = []
    for i in x:
        #parse(i)
        empty_list.append(parse(i))
    return empty_list
    

In [195]:
boweryData['mergeDay'] = [i.day for i in boweryData.date]
boweryData['mergeMonth'] = [i.month for i in boweryData.date]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [196]:
seatgeek.date = parse_time(seatgeek.date)

In [199]:
seatgeek['mergeDay'] = [i.day for i in seatgeek.date]
seatgeek['mergeMonth'] = [i.month for i in seatgeek.date]

In [202]:
# merge bowery and seatgeek
final = pd.merge(boweryData, seatgeek,  how='left', on = ['mergeDay', 'mergeMonth', 'venue'])

In [207]:
# drop false spotify match
final = final[final.artist_name_y != "Kung Fu"]

In [211]:
final['secondary_price']= np.where(final['price'].isnull(), final['primary_price'],
                                   np.where(final['price'] == "No_resale", final['primary_price'], final['price']))




In [213]:
final['mark_up'] = ((final.secondary_price / final.primary_price) -1)*100

In [217]:
final.mark_up.mean()

281.80342592968259

In [377]:
venuesDict = {'Aisle 5': 200,
 'Brooklyn Steel': 1800,
 'Fete Music Hall': 500,
 'Gasa Gasa': 200,
 'Great Scott': 240,
 'Music Hall of Williamsburg': 650,
 'Rams Head Live': 1500,
 'Rough Trade NYC': 300,
 'Royale Boston': 1000,
 'Saturn': 500,
 'Starland Ballroom': 2000,
 'Terminal 5': 3000,
 'Terminal West': 600,
 'The Earl': 300,
 'The National': 1500,
 'The Norva': 1500,
 'The Sinclair': 525,
 'Underground Arts': 600,
 'Variety Playhouse': 1000}

In [404]:
keys = pd.DataFrame(list(venuesDict.keys()))
values = pd.DataFrame(list(venuesDict.values()))
capacity = pd.concat([keys, values], axis=1)
capacity.columns = ['venue', 'capacity']


In [406]:
final = pd.merge(final, capacity,  how='left', on = 'venue')

In [411]:
% store final

Stored 'final' (DataFrame)
